In [1]:
import pandas as pd 
import numpy as np 
import string
import re
import nltk
import random
import xgboost as xgb
from pandas import DataFrame

from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import model_selection
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics


In [2]:
import os
os.chdir(r"") 

rawData = pd.read_csv('webmd.csv')

pd.set_option('display.max_colwidth', -1)

In [3]:
rawData.head()

,Age,Condition,Date,Drug,DrugId,EaseofUse,Effectiveness,Reviews,Satisfaction,Sex,Sides,UsefulCount
0,75 or over,Stuffy Nose,9/21/2014,25dph-7.5peh,146724,5,5,I'm a retired physician and of all the meds I have tried for my allergies (seasonal and not) - this one is the most effective for me. When I first began using this drug some years ago - tiredness as a problem but is not currently.,5,Male,"Drowsiness, dizziness , dry mouth /nose/throat, headache , upset stomach , constipation , or trouble sleeping may occur.",0
1,25-34,Cold Symptoms,1/13/2011,25dph-7.5peh,146724,5,5,cleared me right up even with my throat hurting it went away after taking the medicine,5,Female,"Drowsiness, dizziness , dry mouth /nose/throat, headache , upset stomach , constipation , or trouble sleeping may occur.",1
2,65-74,Other,7/16/2012,warfarin (bulk) 100 % powder,144731,2,3,why did my PTINR go from a normal of 2.5 to over \n100?,3,Female,,0
3,75 or over,Other,9/23/2010,warfarin (bulk) 100 % powder,144731,2,2,FALLING AND DON'T REALISE IT,1,Female,,0
4,35-44,Other,1/6/2009,warfarin (bulk) 100 % powder,144731,1,1,"My grandfather was prescribed this medication (Coumadin) to assist in blood thinning due to a heart and thyroid condition. His primary doctor was aware that he was on an aspirin regiment and still prescribed this medicine, it caused his blood to thin out to much and he ended up internally bleeding to death. If you are going to take this medicine please ask your doctors about possible side effects or drug interactions.",1,Male,,1


In [4]:
rawData.insert(0, 'ID', range(1, 1 + len(rawData)))
rawData.head()

,ID,Age,Condition,Date,Drug,DrugId,EaseofUse,Effectiveness,Reviews,Satisfaction,Sex,Sides,UsefulCount
0,1,75 or over,Stuffy Nose,9/21/2014,25dph-7.5peh,146724,5,5,I'm a retired physician and of all the meds I have tried for my allergies (seasonal and not) - this one is the most effective for me. When I first began using this drug some years ago - tiredness as a problem but is not currently.,5,Male,"Drowsiness, dizziness , dry mouth /nose/throat, headache , upset stomach , constipation , or trouble sleeping may occur.",0
1,2,25-34,Cold Symptoms,1/13/2011,25dph-7.5peh,146724,5,5,cleared me right up even with my throat hurting it went away after taking the medicine,5,Female,"Drowsiness, dizziness , dry mouth /nose/throat, headache , upset stomach , constipation , or trouble sleeping may occur.",1
2,3,65-74,Other,7/16/2012,warfarin (bulk) 100 % powder,144731,2,3,why did my PTINR go from a normal of 2.5 to over \n100?,3,Female,,0
3,4,75 or over,Other,9/23/2010,warfarin (bulk) 100 % powder,144731,2,2,FALLING AND DON'T REALISE IT,1,Female,,0
4,5,35-44,Other,1/6/2009,warfarin (bulk) 100 % powder,144731,1,1,"My grandfather was prescribed this medication (Coumadin) to assist in blood thinning due to a heart and thyroid condition. His primary doctor was aware that he was on an aspirin regiment and still prescribed this medicine, it caused his blood to thin out to much and he ended up internally bleeding to death. If you are going to take this medicine please ask your doctors about possible side effects or drug interactions.",1,Male,,1


In [5]:
len(rawData.Reviews)

362806

In [6]:
 rawData['Reviews'].isnull().sum() #will remove these rows in data cleaning step

37

In [7]:
DataFrame(rawData['Satisfaction'].value_counts())

,Satisfaction
5,111550
1,100901
4,63158
3,51852
2,35342
6,2
10,1


In [8]:
rawData = rawData[["ID","Reviews","Satisfaction","Condition"]]
rawData ['Reviews'] = rawData['Reviews'].str.lower()
rawData.head()

,ID,Reviews,Satisfaction,Condition
0,1,i'm a retired physician and of all the meds i have tried for my allergies (seasonal and not) - this one is the most effective for me. when i first began using this drug some years ago - tiredness as a problem but is not currently.,5,Stuffy Nose
1,2,cleared me right up even with my throat hurting it went away after taking the medicine,5,Cold Symptoms
2,3,why did my ptinr go from a normal of 2.5 to over \n100?,3,Other
3,4,falling and don't realise it,1,Other
4,5,"my grandfather was prescribed this medication (coumadin) to assist in blood thinning due to a heart and thyroid condition. his primary doctor was aware that he was on an aspirin regiment and still prescribed this medicine, it caused his blood to thin out to much and he ended up internally bleeding to death. if you are going to take this medicine please ask your doctors about possible side effects or drug interactions.",1,Other


In [9]:
list = ['High Blood Pressure']
rawData_HBP = rawData[rawData.Condition.isin(list)]
rawData_HBP['Reviews'].count()

23271

In [10]:
DataFrame(rawData.drop(rawData[rawData['Satisfaction'] > 5].index, inplace = True))

""


In [11]:
DataFrame(rawData_HBP['Satisfaction'].value_counts())

,Satisfaction
1,7412
5,5367
3,3922
4,3707
2,2867


In [12]:
rawData_HBP = rawData_HBP[["ID","Reviews","Satisfaction"]]
rawData_HBP['Reviews'] = rawData_HBP['Reviews'].str.lower()
rawData_HBP.reset_index()
rawData_HBP.head()

,ID,Reviews,Satisfaction
3918,3919,i am having bouts of horrible dry coughing.,3
3930,3931,"initially, i was put on lotrel 10/20 for moderate hypertension. i lost weight and improved lifestyle, and bp started being too low, so my dr. lowered dosage to 5/10. i have successfully taken it for years (not really sure if lotrel or generic; seems like it depends on pharmacy and/or changes in my insurance). i do not know if it is related to this med or not, but over the last year, i have developed extreme rash and unbearable itching. dermatologist (i've seen 2 different ones) have prescribed a multitude of creams, moisturizers, steroids, etc. possible diagnoses of contact dermatitis (no way). it changes locations (inside forearms, stomach, back,",4
3939,3940,"this medication caused my blood pressure to elevate after taking for a couple of months. my numbers would be 158/108. i was dehydrated from urinating constantly. swelling of fingers, ankles and feet.",1
3948,3949,,5
3958,3959,"i was on this medication for 2 weeks. had side effects that were similar to the flu plus swelling of the left side of my throat, ear pain. also there was a constant drip of fluid from the left side of my head down to my left ear. i stopped med after 2 weeks. i was treated for a ear infection 1 month later. i was seen in the er for chest pain. saw my primary during those first 2 weeks he felt it was the flu and my imagination. the left side of my head felt like it was closed up. had ringing in my ears and decreased hearing, which was verified by hearing test. the ringing has lessened a lot over the past 2 years. changed primary doctors and now on losartan hct and amlodipine, no side effects and controls bp.",1


In [13]:
indexNames = rawData_HBP[ (rawData_HBP['Satisfaction'] >= 2) & (rawData_HBP['Satisfaction'] <= 3)].index
rawData_HBP.drop(indexNames , inplace=True)

In [14]:
rawData_HBP.head()

,ID,Reviews,Satisfaction
3930,3931,"initially, i was put on lotrel 10/20 for moderate hypertension. i lost weight and improved lifestyle, and bp started being too low, so my dr. lowered dosage to 5/10. i have successfully taken it for years (not really sure if lotrel or generic; seems like it depends on pharmacy and/or changes in my insurance). i do not know if it is related to this med or not, but over the last year, i have developed extreme rash and unbearable itching. dermatologist (i've seen 2 different ones) have prescribed a multitude of creams, moisturizers, steroids, etc. possible diagnoses of contact dermatitis (no way). it changes locations (inside forearms, stomach, back,",4
3939,3940,"this medication caused my blood pressure to elevate after taking for a couple of months. my numbers would be 158/108. i was dehydrated from urinating constantly. swelling of fingers, ankles and feet.",1
3948,3949,,5
3958,3959,"i was on this medication for 2 weeks. had side effects that were similar to the flu plus swelling of the left side of my throat, ear pain. also there was a constant drip of fluid from the left side of my head down to my left ear. i stopped med after 2 weeks. i was treated for a ear infection 1 month later. i was seen in the er for chest pain. saw my primary during those first 2 weeks he felt it was the flu and my imagination. the left side of my head felt like it was closed up. had ringing in my ears and decreased hearing, which was verified by hearing test. the ringing has lessened a lot over the past 2 years. changed primary doctors and now on losartan hct and amlodipine, no side effects and controls bp.",1
3967,3968,i have a scratchy throat that makes me cough all day & night. my cough is wet like water on my lungs. i have leg pain. my knee hurts more than every before. been on it 2 mo. dont like it.,1


In [15]:
rawData_HBP['Satisfaction'] = rawData_HBP['Satisfaction'].replace([5],'Positive')
rawData_HBP['Satisfaction'] = rawData_HBP['Satisfaction'].replace([4],'Positive')
rawData_HBP['Satisfaction'] = rawData_HBP['Satisfaction'].replace([1],'Negative')


In [16]:
rawData_HBP['Satisfaction'].value_counts()

Positive    9074
Negative    7412
Name: Satisfaction, dtype: int64

In [17]:
rawData_HBP.head()

,ID,Reviews,Satisfaction
3930,3931,"initially, i was put on lotrel 10/20 for moderate hypertension. i lost weight and improved lifestyle, and bp started being too low, so my dr. lowered dosage to 5/10. i have successfully taken it for years (not really sure if lotrel or generic; seems like it depends on pharmacy and/or changes in my insurance). i do not know if it is related to this med or not, but over the last year, i have developed extreme rash and unbearable itching. dermatologist (i've seen 2 different ones) have prescribed a multitude of creams, moisturizers, steroids, etc. possible diagnoses of contact dermatitis (no way). it changes locations (inside forearms, stomach, back,",Positive
3939,3940,"this medication caused my blood pressure to elevate after taking for a couple of months. my numbers would be 158/108. i was dehydrated from urinating constantly. swelling of fingers, ankles and feet.",Negative
3948,3949,,Positive
3958,3959,"i was on this medication for 2 weeks. had side effects that were similar to the flu plus swelling of the left side of my throat, ear pain. also there was a constant drip of fluid from the left side of my head down to my left ear. i stopped med after 2 weeks. i was treated for a ear infection 1 month later. i was seen in the er for chest pain. saw my primary during those first 2 weeks he felt it was the flu and my imagination. the left side of my head felt like it was closed up. had ringing in my ears and decreased hearing, which was verified by hearing test. the ringing has lessened a lot over the past 2 years. changed primary doctors and now on losartan hct and amlodipine, no side effects and controls bp.",Negative
3967,3968,i have a scratchy throat that makes me cough all day & night. my cough is wet like water on my lungs. i have leg pain. my knee hurts more than every before. been on it 2 mo. dont like it.,Negative


In [18]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words_english = set(stopwords.words('english')) 
rawData_HBP.dropna(inplace=True)

def english_stop_word_remover(x):
    count = type(x)
    word_tokens = word_tokenize(x) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words_english] 
  
    filtered_sentence = [] 
    
    for w in word_tokens: 
        if w not in stop_words_english: 
            filtered_sentence.append(w) 
    return ' '.join(filtered_sentence)



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pruth\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
rawData_HBP['Reviews'] = rawData_HBP['Reviews'].apply(english_stop_word_remover)

rawData_HBP.head()

,ID,Reviews,Satisfaction
3930,3931,"initially , put lotrel 10/20 moderate hypertension . lost weight improved lifestyle , bp started low , dr. lowered dosage 5/10 . successfully taken years ( really sure lotrel generic ; seems like depends pharmacy and/or changes insurance ) . know related med , last year , developed extreme rash unbearable itching . dermatologist ( 've seen 2 different ones ) prescribed multitude creams , moisturizers , steroids , etc . possible diagnoses contact dermatitis ( way ) . changes locations ( inside forearms , stomach , back ,",Positive
3939,3940,"medication caused blood pressure elevate taking couple months . numbers would 158/108 . dehydrated urinating constantly . swelling fingers , ankles feet .",Negative
3948,3949,,Positive
3958,3959,"medication 2 weeks . side effects similar flu plus swelling left side throat , ear pain . also constant drip fluid left side head left ear . stopped med 2 weeks . treated ear infection 1 month later . seen er chest pain . saw primary first 2 weeks felt flu imagination . left side head felt like closed . ringing ears decreased hearing , verified hearing test . ringing lessened lot past 2 years . changed primary doctors losartan hct amlodipine , side effects controls bp .",Negative
3967,3968,scratchy throat makes cough day & night . cough wet like water lungs . leg pain . knee hurts every . 2 mo . dont like .,Negative


In [20]:
rawData_HBP['Reviews'].count()

16484

In [21]:
nan_value = float("NaN")
#Convert NaN values to empty string

rawData_HBP.replace("", nan_value, inplace=True)

rawData_HBP.dropna(subset = ["Reviews"], inplace=True)
rawData_HBP.head()

,ID,Reviews,Satisfaction
3930,3931,"initially , put lotrel 10/20 moderate hypertension . lost weight improved lifestyle , bp started low , dr. lowered dosage 5/10 . successfully taken years ( really sure lotrel generic ; seems like depends pharmacy and/or changes insurance ) . know related med , last year , developed extreme rash unbearable itching . dermatologist ( 've seen 2 different ones ) prescribed multitude creams , moisturizers , steroids , etc . possible diagnoses contact dermatitis ( way ) . changes locations ( inside forearms , stomach , back ,",Positive
3939,3940,"medication caused blood pressure elevate taking couple months . numbers would 158/108 . dehydrated urinating constantly . swelling fingers , ankles feet .",Negative
3958,3959,"medication 2 weeks . side effects similar flu plus swelling left side throat , ear pain . also constant drip fluid left side head left ear . stopped med 2 weeks . treated ear infection 1 month later . seen er chest pain . saw primary first 2 weeks felt flu imagination . left side head felt like closed . ringing ears decreased hearing , verified hearing test . ringing lessened lot past 2 years . changed primary doctors losartan hct amlodipine , side effects controls bp .",Negative
3967,3968,scratchy throat makes cough day & night . cough wet like water lungs . leg pain . knee hurts every . 2 mo . dont like .,Negative
3977,3978,"since starting lotrel , lab work showing low white blood cell . wonder drug lotrel cause .",Positive


In [22]:
rawData_HBP['Reviews'].count() #there were 16484 reviews before but it seems many reviews had nothing in the review because of which above code removed almost 3k reviews

13697

In [23]:
#removing symbols, links, URLs and punctuations
cleanData = rawData_HBP.dropna(subset=['Reviews'])


cleanData['Reviews'] = cleanData['Reviews'].replace('[^a-zA-Z0-9 ]+', ' ', regex=True) #removes sp char
cleanData['Reviews'] = cleanData['Reviews'].replace('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', ' ', regex=True) #removes single char

cleanData['Reviews'] = cleanData.Reviews.str.strip()
cleanData.head()

,ID,Reviews,Satisfaction
3930,3931,initially put lotrel 10 20 moderate hypertension lost weight improved lifestyle bp started low dr lowered dosage 5 10 successfully taken years really sure lotrel generic seems like depends pharmacy and or changes insurance know related med last year developed extreme rash unbearable itching dermatologist ve seen 2 different ones prescribed multitude creams moisturizers steroids etc possible diagnoses contact dermatitis way changes locations inside forearms stomach back,Positive
3939,3940,medication caused blood pressure elevate taking couple months numbers would 158 108 dehydrated urinating constantly swelling fingers ankles feet,Negative
3958,3959,medication 2 weeks side effects similar flu plus swelling left side throat ear pain also constant drip fluid left side head left ear stopped med 2 weeks treated ear infection 1 month later seen er chest pain saw primary first 2 weeks felt flu imagination left side head felt like closed ringing ears decreased hearing verified hearing test ringing lessened lot past 2 years changed primary doctors losartan hct amlodipine side effects controls bp,Negative
3967,3968,scratchy throat makes cough day night cough wet like water lungs leg pain knee hurts every 2 mo dont like,Negative
3977,3978,since starting lotrel lab work showing low white blood cell wonder drug lotrel cause,Positive


In [24]:
cleanData['Satisfaction'].value_counts()

Negative    7135
Positive    6562
Name: Satisfaction, dtype: int64

In [25]:
#cleanData.to_csv("Path\\clean_data.csv", index= False)

In [26]:
train,test = train_test_split(cleanData,test_size = .25 ,random_state = 112)


In [27]:
train['Satisfaction'].value_counts()


Negative    5366
Positive    4906
Name: Satisfaction, dtype: int64

In [28]:
test['Satisfaction'].value_counts()


Negative    1769
Positive    1656
Name: Satisfaction, dtype: int64

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_tf = count_vect.fit_transform(train.Reviews)
X_train_tf.shape

(10272, 11708)

In [30]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_tf)
X_train_tfidf.shape

(10272, 11708)

In [31]:
#naive bayes
from sklearn.naive_bayes import MultinomialNB
clf= MultinomialNB().fit(X_train_tfidf, train.Satisfaction)

In [32]:
X_test_tf = count_vect.transform(test.Reviews)
X_test_tfidf = tfidf_transformer.transform(X_test_tf)
predicted = clf.predict(X_test_tfidf)

In [33]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(test.Satisfaction, predicted))
print(metrics.classification_report(test.Satisfaction, predicted, target_names = ["negative","positive"]))
metrics.confusion_matrix(test.Satisfaction, predicted)

Accuracy: 0.8335766423357664
              precision    recall  f1-score   support

    negative       0.81      0.89      0.85      1769
    positive       0.87      0.77      0.82      1656

    accuracy                           0.83      3425
   macro avg       0.84      0.83      0.83      3425
weighted avg       0.84      0.83      0.83      3425



array([[1580,  189],
       [ 381, 1275]], dtype=int64)

In [34]:
#SVM with linear kernel
from sklearn import svm
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf, train.Satisfaction)

SVC(gamma='auto', kernel='linear')

In [35]:
predicted_SVM = SVM.predict(X_test_tfidf)

In [36]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(test.Satisfaction, predicted_SVM))
print(metrics.classification_report(test.Satisfaction, predicted_SVM, target_names = ["negative","positive"]))
metrics.confusion_matrix(test.Satisfaction, predicted_SVM)

Accuracy: 0.8557664233576643
              precision    recall  f1-score   support

    negative       0.85      0.87      0.86      1769
    positive       0.86      0.84      0.85      1656

    accuracy                           0.86      3425
   macro avg       0.86      0.86      0.86      3425
weighted avg       0.86      0.86      0.86      3425



array([[1545,  224],
       [ 270, 1386]], dtype=int64)

In [37]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


RFclassifier = RandomForestClassifier()
RFclassifier.fit(X_train_tfidf, train.Satisfaction)

RandomForestClassifier()

In [38]:
predicted_RF = RFclassifier.predict(X_test_tfidf)

In [39]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(test.Satisfaction, predicted_RF))
print(metrics.classification_report(test.Satisfaction, predicted_RF, target_names = ["negative","positive"]))
metrics.confusion_matrix(test.Satisfaction, predicted_RF)

Accuracy: 0.8633576642335766
              precision    recall  f1-score   support

    negative       0.85      0.89      0.87      1769
    positive       0.87      0.84      0.86      1656

    accuracy                           0.86      3425
   macro avg       0.86      0.86      0.86      3425
weighted avg       0.86      0.86      0.86      3425



array([[1570,  199],
       [ 269, 1387]], dtype=int64)

In [40]:
# adaboost

# base estimator
tree = DecisionTreeClassifier(max_depth=13)

# adaboost with the tree as base estimator
adaboost_model_1 = AdaBoostClassifier(
    base_estimator=tree,
    n_estimators=100,
    learning_rate=1,
    random_state=42,
    algorithm="SAMME")

In [41]:
adaboost_model_1.fit(X_train_tfidf, train.Satisfaction)
adaboost_predictions = adaboost_model_1.predict(X_test_tfidf) 

In [42]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(test.Satisfaction, adaboost_predictions))
print(metrics.classification_report(test.Satisfaction, adaboost_predictions, target_names = ["negative","positive"]))
metrics.confusion_matrix(test.Satisfaction, adaboost_predictions)

Accuracy: 0.8613138686131386
              precision    recall  f1-score   support

    negative       0.86      0.88      0.87      1769
    positive       0.87      0.84      0.85      1656

    accuracy                           0.86      3425
   macro avg       0.86      0.86      0.86      3425
weighted avg       0.86      0.86      0.86      3425



array([[1555,  214],
       [ 261, 1395]], dtype=int64)

In [43]:
#Both Random Forest and ada boost seems to perform the same but I will go ahead with the RF model for the deployment in production